In [1]:
import networkx as nx # version 2.2
import matplotlib.pyplot as plt
import re
import cvxpy as cp
import operator #to sort elements in a list of tuples
import itertools
import math
import numpy as np
import os
import sys
import time

import Init_NetRate as Init
import cvxpy as cp
import Cascade_generation_functions_NetRate as Gen
import CVX_functions as cvx

In [2]:
network_file_name = "./Graph_test_SG.txt"
cascade_file_name = "./Cascade_test_SG.txt"
window = 10
eps = 0.0005
alpha_max = 10
model=0
beta = 1

In [3]:
G_true = Init.Load_ground_truth(network_file_name)
G_star, DAG_C_dic = Init.Init(cascade_file_name)
N = G_true.number_of_nodes()

All nodes were read


In [4]:
# G_true = Gen.Generate_random_graph(10,20)
# Cascades = Gen.Generate_all_cascades(G_true,-100,window,model,beta)
# Gen.Save_cascade_to_file("Cascade_test_ADMM.txt",Cascades,G_true)
# Gen.Save_graph_to_file("Graph_test_ADMM.txt",G_true)
# G_star,DAG_C_dic = Init.Init("Cascade_test_ADMM.txt")
# N = G_true.number_of_nodes()

In [5]:
(A_pot,A_bad),num_casc_per_node = cvx.Create_matrices_ADMM(G_true,DAG_C_dic,window)
A_hat = np.zeros((N,N))
u = 3

# Start ADMM

In [ ]:
t_start_global = time.time()
dic_of_obj_per_node_per_iter = {}
obj_per_node = []
for i in G_true.nodes :
    t_start_node_i = time.time()
    dic_of_obj_per_node_per_iter[i] = []
    try :
        cascade_list = num_casc_per_node[i]
    except KeyError :
        A_hat[:,i] =0
        continue
    rho = np.zeros((len(DAG_C_dic),1))
    a_0 = np.ones(N)
    S_i = np.zeros((len(DAG_C_dic),1))
    print("Node : ",i)
    for k in range(0,20) :
#         print("iteration : ",k)
        '''
        update alpha_i
        '''
        a_hat = cp.Variable(N)
        obj = 0
        constr = [a_hat>=0]
        for j in G_true.nodes :
            if A_pot[j,i]>0:
                obj += a_hat[j]*(A_pot[j,i]+A_bad[j,i]) # to do double check the signe
            else :
                constr.append(a_hat[j]==0)
        for casc in cascade_list : # not sure about this one. It may be DAG_C_dic instead of cascade_list
            c = DAG_C_dic[casc]
            tmp = 0
            S_0 = 0
            if i in c.nodes:
                parent_i_c = list(c.predecessors(i))
                for papa in parent_i_c :
                    tmp += a_hat[papa]
                    S_0 +=a_0[papa] 
            obj += rho[casc]*tmp # to do double check the signe
            if not all(s_i==0 for s_i in S_i):
                obj += u/2 * cp.power((S_i[casc]-tmp),2)
            else :
                obj += u/2 * cp.power((S_0-tmp),2) # same

        objf = cp.Minimize(obj)
        prob = cp.Problem(objf,constr)
        res = prob.solve()
        if prob.status!="optimal":
            print(prob.status)
        a_k = a_hat.value
        for index,alpha in enumerate(a_k):
            if alpha<eps:
                a_k[index]=eps
            if alpha>alpha_max :
                a_k[index] = alpha_max
        '''
        update S_i and rho for each cascades via the closed form formula and the gradient descent respectively
        '''
        for cascs in DAG_C_dic :
            c = DAG_C_dic[cascs]
            Malpha = 0
            if i in c.nodes :
                parent_i_c = list(c.predecessors(i))
                for papa in parent_i_c :
                    Malpha += a_k[papa]
            sqrt_delta = math.sqrt((rho[cascs]+Malpha)**2 + 4*u)
            S_i[cascs] = ((rho[cascs]+Malpha)+sqrt_delta)/(2*u)
            if S_i[cascs]<0 :
                print("Huston Huston we have a probleme")
            rho[cascs] = rho[cascs]-u*(S_i[cascs]-Malpha)
            
        '''compute the objective function for node i in iteration k'''
        obj_i_k = 0
        for c in DAG_C_dic :
            cascade = DAG_C_dic[c]
            sum_tmp = 0
            for j in cascade.nodes():
                t_j = cascade.nodes[j]["time"]
                if a_k[j]<0:
                        a_k[j]=0
                if (j,i) in cascade.edges():
                    t_i = cascade.node[i]["time"]
                    obj_i_k += -a_k[j]*(t_i-t_j)
                    if a_k[j]<eps:
                        sum_tmp += eps
                    else:
                        sum_tmp += a_k[j]
                if j in cascade.nodes() and i not in cascade.nodes():
                    obj_i_k += -a_k[j]*(window-t_j)
            if sum_tmp >0:
                obj_i_k += math.log(sum_tmp)
        dic_of_obj_per_node_per_iter[i].append(obj_i_k)
    A_hat[:,i] = a_k
    obj_per_node.append(obj_i_k)
    print("Total objective function value is : ",sum(obj_per_node))
    t_end_node_i = time.time()
    print("computation time for node i : ", t_end_node_i-t_start_node_i)
t_end_global = time.time()
print("total computation time : ",t_end_global-t_start_global) 

Node :  0
Total objective function value is :  -219.12913578930005
computation time for node i :  22.55497455596924
Node :  1
Total objective function value is :  -315.2384994969866
computation time for node i :  11.200297594070435
Node :  2
Total objective function value is :  -1039.2721644551295
computation time for node i :  122.56139373779297
Node :  3
Total objective function value is :  -1165.2701447626282
computation time for node i :  10.83068060874939
Node :  4
Total objective function value is :  -2025.5112807428873
computation time for node i :  143.60041117668152
Node :  5
Total objective function value is :  -2837.1032459405237
computation time for node i :  129.99328303337097
Node :  6
Total objective function value is :  -2956.582784402341
computation time for node i :  9.942332029342651
Node :  7
Total objective function value is :  -3543.39718754573
computation time for node i :  116.11854910850525
Node :  8
Total objective function value is :  -4504.867433675138
compu

Total objective function value is :  -42274.633782091354
computation time for node i :  95.81569981575012
Node :  73
Total objective function value is :  -43095.37185558575
computation time for node i :  110.71150302886963
Node :  74
Total objective function value is :  -43517.64239388737
computation time for node i :  61.583460092544556
Node :  75
Total objective function value is :  -43992.228696135375
computation time for node i :  76.7299370765686
Node :  76
Total objective function value is :  -45065.4821493421
computation time for node i :  126.05709719657898
Node :  77
Total objective function value is :  -45323.49969142366
computation time for node i :  29.456066370010376
Node :  78
Total objective function value is :  -46161.58960668121
computation time for node i :  112.06603026390076
Node :  79
Total objective function value is :  -46711.75818640412
computation time for node i :  90.86486864089966
Node :  80
Total objective function value is :  -47187.4565325317
computation 

Total objective function value is :  -79514.07230752813
computation time for node i :  98.81814312934875
Node :  144
Total objective function value is :  -79982.1132300647
computation time for node i :  92.05183744430542
Node :  145
Total objective function value is :  -80766.20253621672
computation time for node i :  106.27837204933167
Node :  146
Total objective function value is :  -81592.00257842154
computation time for node i :  119.79757833480835
Node :  147
Total objective function value is :  -82657.95121968121
computation time for node i :  109.87025666236877
Node :  148
Total objective function value is :  -82754.27378278307
computation time for node i :  8.460475206375122
Node :  149
Total objective function value is :  -82888.06449503801
computation time for node i :  9.343733310699463
Node :  151
Total objective function value is :  -82984.8681920842
computation time for node i :  8.639775276184082
Node :  152
Total objective function value is :  -83702.99068783957
computa

In [ ]:
'''
Objective function for node i only
'''
obj = 0
for c in DAG_C_dic :
    cascade = DAG_C_dic[c]
    sum_tmp = 0
    for j in cascade.nodes():
        t_j = cascade.nodes[j]["time"]
        if (j,i) in cascade.edges():
            print(A_hat[j,i])
            t_i = cascade.node[i]["time"]
            obj += -A_hat[j,i]*(t_i-t_j)
            sum_tmp += A_hat[j,i]
        if j in cascade.nodes() and i not in cascade.nodes():
            obj += -A_hat[j,i]*(window-t_j)
    if sum_tmp !=0:
        obj += math.log(sum_tmp)
    

In [19]:
'''
Precision and recall computation
'''
correct = 0
edge_G_star = list(G_star.edges())
G_star.remove_edges_from(edge_G_star)
for i in range(0,G_true.number_of_nodes()):
    for j in range(0,G_true.number_of_nodes()):
        if A_hat[i,j] >eps:
            G_star.add_edge(i,j)
            if (i,j) in G_true.edges():
                correct +=1
#                 print("edge ",(i,j))
#                 print("alpha is ",A_hat[i,j])
print("Precision :",correct/G_star.number_of_edges())
print("Recall :",correct/G_true.number_of_edges())
print("Number of correct infered edges : ",correct)
print("Number of edges in G_hat :",G_star.number_of_edges())
print("Number of edges in the true network ",G_true.number_of_edges())
        

Precision : 1.0
Recall : 0.9
Number of correct infered edges :  18
Number of edges in G_hat : 18
Number of edges in the true network  20


In [17]:
'''
mean square error
'''
mse = 0
nb_correcte_edges=0
for edge in G_true.edges():
    if edge in G_star.edges():
        nb_correcte_edges +=1
        try:
            true_alpha = G_true.edges[edge]["weight"][0]
        except TypeError:
            true_alpha = G_true.edges[edge]["weight"]
        approx_alpha = A_hat[edge[0],edge[1]]
        mse += pow((true_alpha-approx_alpha),2) # mean square error
#         print("For edge " + str(edge)+ " the true weight is : " + str(true_alpha))
#         print("For edge " + str(edge)+ " the approx weight is : " + str(approx_alpha))
try :
    mse = mse/nb_correcte_edges
except ZeroDivisionError:
    print ("There is no correct edge")
print(" Mean square error is : ",mse)

 Mean square error is :  0.13578798430827993


In [ ]:
G_true.edges[0,152]